In [1]:
import os
import sys

# Set the environment variable for PySpark to the current path of the Python interpreter (sys.executable)

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
# Import the required libraries and modules

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, IntegerType
from pyspark.mllib.linalg.distributed import RowMatrix, IndexedRowMatrix
from pyspark.ml.regression import LinearRegression
from pyspark.ml.functions import array_to_vector
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
from pyspark.mllib.linalg import DenseMatrix
import time
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.linalg import lu, lu_factor, lu_solve # is used for LU decomposition

# Initialize

In [36]:
# Initialization and configuration of the Spark session.

# Set the master node for Spark (running locally with 4 cores on a Spark standalone cluster).
# Using .appName() to set the name for the application, which will be displayed on the user interface.
# With .getOrCreate() an existing Spark session is retrieved, if none exists, one is created
spark = SparkSession.builder \
                    .master('local[4]') \
                    .appName('DBiBD') \
                    .getOrCreate()

In [37]:
# number of rows/samples
n=1000

# number of colums/features
k=3

In [38]:
# Generate random coefficients (betas)
betas=(np.random.rand(k)-0.5)*20
betas

array([ 6.83281826, -6.73397765,  1.35729456])

In [39]:
# Generate random covariances (cov)
cov=(np.random.rand(k)-0.5)*20
cov

array([ 1.19605922,  0.47050515, -7.83438589])

# generate dataset

In [40]:
# Generate an RDD with n vectors, each containing k entries, where each entry is generated from a standard-normal distribution
# With spark.sparkContext the interface to the Spark cluster is realized. This allows to execute all necessary operations on the cluster
data = RandomRDDs.normalVectorRDD(spark.sparkContext, n, k)

In [41]:
# Applying this function yields an rdd where the first element is a vector with a moderate covariance structure and some added noise 
# for a more realistic setting, while the second element is the target variable.
def createRow(noise):
    x=[]
    x.append(noise[0])
    for i in range(1,len(noise)):
        x.append((x[0]*cov[i])+noise[i])
    x= [float(a) for a in x]
    
    y=0
    for i in range(0,len(x)):
        y+=x[i]*betas[i]
    
    return x,float(y)
    

In [42]:
# Application of the createRow function to each element of the RDD (data)
data=data.map(createRow)

## Covariance

In [43]:
# Create a RowMatrix from the vectors of the RDD (data).
# The RowMatrix offers the advantage that it is optimized for the application of operations from the linear algebra. With it an efficient
# implementation of the calculations can be ensured without the need for additional effort to bring the data into the required format.
dataMatrix=RowMatrix(data.map(lambda x : x[0]))

# Calculation of covariance matrix and conversion to array
dataMatrix.computeCovariance().toArray()

array([[ 1.0606416 ,  0.48895633, -8.37592827],
       [ 0.48895633,  1.20720599, -3.85092226],
       [-8.37592827, -3.85092226, 67.19110856]])

In [44]:
# A schema is used to define the structure of a data frame.
# The data frame contains the columns features and y.
# The features column consists of an array of floats per row. It shows the values of the independent variables
# The y column has a single float value per row and describes the dependent variable.
schema = StructType([       
    StructField('features', ArrayType(FloatType()), True),
    StructField('y', FloatType(), True)
])

# Create the data frame df using the schema defined at the beginning
dataDF=data.toDF(schema=schema)
# Display of the data frame
dataDF.show(truncate=False)

+--------------------------------------+-----------+
|features                              |y          |
+--------------------------------------+-----------+
|[0.099376045, -0.7133142, -0.9548688] |4.186422   |
|[0.76255125, 0.19460212, -6.969095]   |-5.5591874 |
|[-1.2307073, -1.5058644, 9.823388]    |15.064489  |
|[-0.87878966, 0.29600775, 6.2963905]  |0.5481366  |
|[0.57823384, 1.1535741, -3.0455763]   |-7.95092   |
|[-0.18514186, -0.23748638, 1.4703904] |2.32994    |
|[-0.10045725, 0.5425021, 1.9396323]   |-1.706951  |
|[0.0780959, 0.2928381, -0.23282509]   |-1.7543622 |
|[-2.037761, 0.7556917, 16.964283]     |4.0130663  |
|[2.2538111, 1.0648041, -18.144684]    |-16.398167 |
|[0.59066653, -0.056806386, -2.4121084]|1.1445082  |
|[0.55334085, 0.98757243, -3.123839]   |-7.1093826 |
|[-0.09503667, 0.8405007, 0.5635708]   |-5.5443497 |
|[0.8281525, 1.0089866, -6.2490616]    |-9.617695  |
|[-0.21122994, 1.2439487, 2.9526653]   |-5.8123817 |
|[0.53106874, 1.8474259, -2.303149]    |-11.93

# PySpark Linear Regression

In [45]:
# Start the timer
start_time = time.time()

# Create a LinearRegression instance to perform the linear regression
lr = LinearRegression(featuresCol="features", labelCol="y", predictionCol="pred_y")

# Calculate the linear model. To use the method correctly, column features is converted to a vector format
lr_model = lr.fit(dataDF.withColumn("features",array_to_vector('features')))

# Finally, the results of the OLS estimation, the true betas and the total execution time are output
print("PySpark OLS: %s seconds" % (time.time() - start_time))
print("real values:\t\t",betas.round(6))
print("predicted values:\t",lr_model.coefficients.round(6))

PySpark OLS: 9.319233655929565 seconds
real values:		 [ 6.832818 -6.733978  1.357295]
predicted values:	 [ 6.832818 -6.733978  1.357295]


# QR

In [46]:
# Start the timer
start_time = time.time()

# Execute the QR decomposition on the dataMatrix using the tallSkinnyQR method by calculating both matrices Q and R (True).
QR=dataMatrix.tallSkinnyQR(True)

# Output of the Q-matrix, the number of rows and columns 
print(QR.Q, QR.Q.numRows(),"x",QR.Q.numCols())

# Print the R matrix.
print(QR.R)

<pyspark.mllib.linalg.distributed.RowMatrix object at 0x0000016CCB980C40> 1000 x 3
DenseMatrix([[  32.55140289,   15.000607  , -257.06243852],
             [   0.        ,   31.35924019,    0.37105258],
             [   0.        ,    0.        ,   32.33428266]])


In [47]:
# Conversion of the R matrix into a Numpy matrix
R=np.asmatrix(QR.R.toArray())

# Calculation of the inverse of R
R_inv=np.linalg.inv(R)

# Output of the inverse
R_inv

matrix([[ 0.03072064, -0.01469514,  0.24440239],
        [ 0.        ,  0.03188853, -0.00036594],
        [ 0.        ,  0.        ,  0.03092693]])

In [48]:
# Creating a CoordinateMatrix (cm) with the Q matrix (QR.Q) via "MatrixEntry" objects
cm = CoordinateMatrix(
    QR.Q.rows.zipWithIndex().flatMap(
        lambda x: [MatrixEntry(x[1], j, v) for j, v in enumerate(x[0])]
    )
)
# Build the transposed Q-matrix (Q_T) using the CoordinateMatrix (cm) and convert it to a RowMatrix (toRowMatrix())
Q_T=cm.transpose().toRowMatrix()

In [49]:
# The DenseMatrix (y) is built with the parameters n, 1 and the values from the one-dimensional vector.
# The one-dimensional vector is created from the "y" column of the data frame dataDF by converting it to a Numpy array.

y=DenseMatrix(n,1,dataDF.select("y").toPandas().to_numpy().ravel())

## Results

In [50]:
# Performing the matrix multiplications

# Q_T and y are multiplied with the multiply function
# With rows.collect() the calculations of the Spark driver nodes are returned to a local data structure, in this case a Python list
step1 = Q_T.multiply(y).rows.collect() 

# Convert step1 to a numpy array
step1 = np.array(step1)  

# Calculate the betas
step2 = np.matmul(R_inv, step1) 

# Finally, the results of the OLS estimation, the true betas and the total execution time are output
print("QR OLS: %s seconds" % (time.time() - start_time))
print("real values:\t\t", betas.round(6))
print("QR predicted values:\t", step2.ravel().round(6)[0])

QR OLS: 34.09499263763428 seconds
real values:		 [ 6.832818 -6.733978  1.357295]
QR predicted values:	 [ 6.832818 -6.733978  1.357295]


# SVD

In [51]:
# Start the timer
start_time = time.time()

# Compute the singular value decomposition (SVD) of the data matrix dataMatrix with k singular values.
# With computeU = True the both matrices U and V are calculated.
svd=dataMatrix.computeSVD(k,computeU=True)

# Output of the singular values
print(svd.s)

# Output of the matrix V
print(svd.V)

# Output of the matrix U and the properties of it
print(svd.U, svd.U.numRows(),"x",svd.U.numCols())

[261.52173163318855,31.38418119595542,4.0214241975753655]
DenseMatrix([[-0.1235301 , -0.00194794,  0.99233891],
             [-0.05758562, -0.99829884, -0.00912812],
             [ 0.99066857, -0.05827205,  0.12320778]])
<pyspark.mllib.linalg.distributed.RowMatrix object at 0x0000016CCDEA9A60> 1000 x 3


In [52]:
# Creating a CoordinateMatrix (cm) with the U-matrix (svd.U) via "MatrixEntry" objects
cm = CoordinateMatrix(
    svd.U.rows.zipWithIndex().flatMap(
        lambda x: [MatrixEntry(x[1], j, v) for j, v in enumerate(x[0])]
    )
)
# Build the transposed U-matrix (U_T) using the CoordinateMatrix (cm) and convert it to a RowMatrix (toRowMatrix())
U_T=cm.transpose().toRowMatrix()

## Results

In [53]:
# U_T and y are multiplied with the multiply function
# With rows.collect() the calculations of the Spark driver nodes are returned to a local data structure, in this case a Python list
step1=U_T.multiply(y).rows.collect()

# Calculate the OLS estimator according to calculation rule
step2=(np.array(step1).ravel()/svd.s)
v=np.matrix(svd.V.toArray())
SVD_coeeffs=(v @ step2).ravel()

# Finally, the results of the OLS estimation, the true betas and the total execution time are output
print("SVD OLS: %s seconds" % (time.time() - start_time))
print("real values:\t\t",betas.round(6))
print("SVD predicted values:\t",SVD_coeeffs.round(6)[0])

SVD OLS: 31.016135692596436 seconds
real values:		 [ 6.832818 -6.733978  1.357295]
SVD predicted values:	 [ 6.832818 -6.733978  1.357295]


# LU
als Data Paralleism Ansatz da keine LU Funktion in PySpark enthalten

In [54]:
# Start the timer
start_time = time.time()

In [55]:
def luSpark(part):
    # Die Größe der Partition ermitteln
    partition_n = len(part)
    
    # Array Features in nparray schreiben damit die struktur mit .T transponiert werden kann
    lufeatures = np.array([b for b in part["features"].to_numpy()])
    ypanda = part["y"].to_numpy()

    # Durchführen der LU-Composition
    LUtemp, pivtemp = lu_factor(lufeatures.T @ lufeatures)
    lubetas = lu_solve((LUtemp, pivtemp), lufeatures.T @ ypanda)
    
    # Eine DataFrame mit den geschätzten Betas und der Anzahl der Beobachtungen erstellen
    return pd.DataFrame({"betas": [lubetas], "sampleCounts": [partition_n]})

In [56]:
# Define the scheme
schemaUDF=StructType([
    StructField("betas",ArrayType(FloatType())),
    StructField("sampleCounts",IntegerType())
])

In [57]:
# Apply the "luSpark" function to the partition groups of the "dataDF" DataFrame
# Calculate and average the coefficients for each partition
LU_res = dataDF.groupBy(F.spark_partition_id()).applyInPandas(luSpark,schema=schemaUDF)
LU_res.show(truncate=False)

+----------------------------------+------------+
|betas                             |sampleCounts|
+----------------------------------+------------+
|[6.8325405, -6.7339754, 1.35726]  |250         |
|[6.8327565, -6.7339783, 1.3572867]|250         |
|[6.832314, -6.73398, 1.3572307]   |250         |
|[6.833318, -6.7339807, 1.357357]  |250         |
+----------------------------------+------------+



In [58]:
# calculate weighted average of estimated betas for all partitions
LU_betas = pd.DataFrame(LU_res.rdd.map(lambda x : [(x["sampleCounts"]/n) * xi for xi in x["betas"]]).collect()).sum().to_numpy()

In [59]:
# Finally, the results of the OLS estimation, the true betas and the total execution time are output
print("LU OLS: %s seconds" % (time.time() - start_time))
print("real values:\t\t",betas.round(6))
print("LU predicted values :\t",LU_betas.round(6))

LU OLS: 12.475306749343872 seconds
real values:		 [ 6.832818 -6.733978  1.357295]
LU predicted values :	 [ 6.832732 -6.733979  1.357284]


In [60]:
# End Spark Session
spark.stop()